In [1]:
# !pip install pymed
# ! pip install nltk
# import nltk
# nltk.download('stopwords')
# nltk.download('punkt')
# !pip install biopython
# !pip install ahocorapy
# !pip install pandas
# !pip install wordcloud
# !pip install matplotlib

## Search PUBMED

In [3]:
def pubMedSearcher(search_term,max_count):
    from pymed import PubMed
    pubmed = PubMed(tool="PubMedSearcher", email="myemail@ccc.com")
    search_term = search_term.rstrip(';')
    if ';' in search_term:
        search_term = search_term.split(";")

        query = ""
        for i in range(len(search_term)-1):
            query+=f"({search_term[i]}) AND "
        query = query+f"({search_term[i+1]})"
    else:
        query = search_term
    print(query)
    ## PUT YOUR SEARCH TERM HERE ##
    results = pubmed.query(query, max_results=max_count)
    articleList = []
    articleInfo = []
    
    for article in results:
    # Print the type of object we've found (can be either PubMedBookArticle or PubMedArticle).
    # We need to convert it to dictionary with available function
        articleDict = article.toDict()
        articleList.append(articleDict)

    # Generate list of dict records which will hold all article details that could be fetch from PUBMED API
    for article in articleList:
    #Sometimes article['pubmed_id'] contains list separated with comma - take first pubmedId in that list - thats article pubmedId
        pubmedId = article['pubmed_id'].partition('\n')[0]
        # Append article info to dictionary 
        articleInfo.append({u'pubmed_id':pubmedId,
                           u'title':article['title'] if 'title' in article else [],
                           u'keywords':article['keywords'] if 'keywords' in article else [],
                           u'journal':article['journal'] if 'journal' in article else [],
                           u'abstract':article['abstract'] if 'abstract' in article else [],
                           u'conclusions':article['conclusions'] if 'conclusions' in article else [],
                           u'methods':article['methods'] if 'methods' in article else [],
                           u'results': article['results'] if 'results' in article else [],
                           u'copyrights':article['copyrights'] if 'copyrights' in article else [],
                           u'doi':article['doi'] if 'doi' in article else [],
                           u'publication_date':article['publication_date'] if 'publication_date' in article else [], 
                           u'authors':article['authors'] if 'authors' in article else []})

#     # Generate Pandas DataFrame from list of dictionaries
#     articlesPD = pd.DataFrame.from_dict(articleInfo)
#     export_csv = df.to_csv (r'C:\Users\YourUsernam\Desktop\export_dataframe.csv', index = None, header=True) 

#     #Print first 10 rows of dataframe
#     print(articlesPD.head(10))
    return articleInfo

In [2]:
# search = 'affinity'
# articles = pubMedSearcher(search,50)
# len(articles)

## extract abstract keywords

In [6]:
def get_tree():
    import re

    mesh_keywords = []
    dmeshFile = 'd2021.bin'
    qmeshFile = 'q2021.bin'
    cmeshFile = 'c2021.bin'
    cdmeshFile = 'c2021_disease.bin'
    
    # descriptors
    with open(dmeshFile, mode='rb') as file:
        dmesh = file.readlines()
    for line in dmesh:
        meshTerm = re.search(b'MH = (.+)$', line)
        if meshTerm:
            term = meshTerm.group(1)
            mesh_keywords.append(term.decode('utf-8'))

    # qualifiers
    with open(qmeshFile, mode='rb') as file:
        qmesh = file.readlines()
    for line in qmesh:
        meshTerm = re.search(b'SH = (.+)$', line)
        if meshTerm:
            term = meshTerm.group(1)
            mesh_keywords.append(term.decode('utf-8'))

    # Supplementary Records
    with open(cmeshFile, mode='rb') as file:
        cmesh = file.readlines()
    for line in cmesh:
        meshTerm = re.search(b'NM = (.+)$', line)
        if meshTerm:
            term = meshTerm.group(1)
            mesh_keywords.append(term.decode('utf-8'))

    # Supplementary Records - Diseases
    with open(cdmeshFile, mode='rb') as file:
        cdmesh = file.readlines()
    for line in cdmesh:
        meshTerm = re.search(b'NM  = (.+)$', line)
        if meshTerm:
            term = meshTerm.group(1)
            mesh_keywords.append(term.decode('utf-8'))

    from ahocorapy.keywordtree import KeywordTree
    kwtree = KeywordTree(case_insensitive=True)

    for kw in mesh_keywords:
        kwtree.add(kw)

    kwtree.finalize()
    return kwtree

def get_abstract_keywords(kwtree,textblob):
    from nltk.corpus import stopwords
    stop_words = set(stopwords.words('english'))
    if textblob:
        results = kwtree.search_all(textblob)
        keywords = []
        for res in results:
            if res is not None and res[0] not in stop_words:
                keywords.append(res[0])
        return list(set(keywords))
    else:
        return []

kwtree = get_tree()

In [7]:
len(articles)

13367

In [8]:
# abstract_keywords = {}
# for paper in articles:
#     abstract_keywords[str(paper['pubmed_id'])] = get_abstract_keywords(kwtree,paper['abstract'])
#     abstract_keywords[str(paper['pubmed_id'])].extend(get_abstract_keywords(kwtree,paper['title']))
#     abstract_keywords[str(paper['pubmed_id'])] = list(set(abstract_keywords[str(paper['pubmed_id'])]))
    
# len(abstract_keywords)

## extract related keywords

In [9]:
def get_related_keywords(papers):
    keywords_related = {}
    for paper in papers:
        keywords_related[str(paper['pubmed_id'])] = paper['keywords']
    return keywords_related


In [10]:
# related_keywords = get_related_keywords(articles)
# len(related_keywords)

## extract predefined keywords

In [11]:
import pandas as pd

def get_fixed_tree():
    from ahocorapy.keywordtree import KeywordTree
    mesh_keywords = []
    File = 'master keywords list.csv'
    keywords = pd.read_csv(File,names =['keyword'])['keyword'].to_list()
    kwtree = KeywordTree(case_insensitive=True)
    for kw in keywords:
        kwtree.add(kw)
    kwtree.finalize()
    return kwtree

def get_predefined_keywords(kwtree,textblob):
    from nltk.corpus import stopwords
    stop_words = set(stopwords.words('english'))
    if textblob:
        textblob = textblob.replace('-',' ')
        results = kwtree.search_all(textblob)
        keywords = []
        for res in results:
            if res is not None and res[0] not in stop_words:
                keywords.append(res[0].lower())
        return list(set(keywords))
    else:
        return []

pkwtree = get_fixed_tree()

In [12]:
# predefined_keywords = {}
# for paper in articles:
#     predefined_keywords[str(paper['pubmed_id'])] = get_predefined_keywords(pkwtree,paper['abstract'])
#     predefined_keywords[str(paper['pubmed_id'])].extend(get_predefined_keywords(pkwtree,paper['title']))
#     predefined_keywords[str(paper['pubmed_id'])] = list(set(predefined_keywords[str(paper['pubmed_id'])]))
# len(predefined_keywords)

## extract Expanded keywords

In [13]:
import json
def paper_keywords_mesh(papers):
    mesh_keywrds_extracted = {}
    for paper in papers:
        mesh_keywrds_extracted[paper['pubmed_id']] = paper['mesh_terms']
    return mesh_keywrds_extracted


def get_expanded_keywords(mesh_database, mesh_keywrds_extracted):
    expanded_keywords = {}
    for pmid, keywords in mesh_keywrds_extracted.items():
        expanded_keywords[pmid] = []
        for keyword in keywords:
            if keyword[0] in mesh_database.keys():
                for key, value in mesh_database[keyword[0]].items():
                    if key in ['subheads', 'subheads_synonyms', 'synonyms', 'cross_reference', 'pharmalogical_actions',
                               'synonym_combination'] and value:
                        expanded_keywords[pmid].extend(value)
        expanded_keywords[pmid] = list(set(expanded_keywords[pmid]))
    return expanded_keywords

In [14]:
with open('mesh_database.json','r') as f:
    mesh_database = json.load(f)
    

# mesh_keywrds_extracted = paper_keywords_mesh(papers)
# expanded_keywords = get_expanded_keywords(mesh_database, mesh_keywrds_extracted)

# len(expanded_keywords)

## Keyword Generation

1. predefined keywords - keywords found directly from the master list apart from the search term
2. abstract keywords - keywords extracted from abstract and title of the paper using a mesh keyword extractor
3. related keywords- keywords extracted from the pubmed response dict in keyword key
4. expanded keywords - keywords extracted from mesh term related [descriptors,qualifiers and Supplementary Chemical Records](https://www.nlm.nih.gov/mesh/intro_record_types.html)

In [4]:
import json
from Bio import Entrez

def related_keywords_mesh(papers):
    mesh_kewords_related = {}
    for paper in papers:
        mesh_kewords_related[paper['title']] = paper['mesh_terms']
    return mesh_kewords_related

import json
import dateutil.parser

# custom Decoder
def DecodeDateTime(Dict):
    if 'publication_date' in Dict and Dict["publication_date"]!=None:
        Dict["publication_date"] = dateutil.parser.parse(Dict["publication_date"])
        return Dict
    else:
        return Dict

# use of object_hook
with open('cache_articles.json', 'r') as f:
    decoded_cache_articles = json.load(f,object_hook=DecodeDateTime)
    

In [21]:
%%time
search = 'ionic liquids'
articles = pubMedSearcher(search,15000)
len(articles)

search_term = "virus capsid;virus-like particles;Protein A"
searchterms = search_term.split(";")
print(searchterms)
search_database = []

for st in searchterms:
    if st in decoded_cache_articles:
        search_database.extend(decoded_cache_articles[st])
articles = search_database

len(articles)

Wall time: 0 ns


13367

In [15]:
%%time
abstract_keywords = {}
for paper in articles:
    abstract_keywords[str(paper['pubmed_id'])] = get_abstract_keywords(kwtree,paper['abstract'])
    abstract_keywords[str(paper['pubmed_id'])].extend(get_abstract_keywords(kwtree,paper['title']))
    abstract_keywords[str(paper['pubmed_id'])] = list(set(abstract_keywords[str(paper['pubmed_id'])]))
    
len(abstract_keywords)

Wall time: 44.1 s


4749

In [16]:
%%time
related_keywords = get_related_keywords(articles)
len(related_keywords)

Wall time: 20.1 ms


4749

In [17]:
%%time
predefined_keywords = {}
for paper in articles:
    predefined_keywords[str(paper['pubmed_id'])] = get_predefined_keywords(pkwtree,paper['abstract'])
    predefined_keywords[str(paper['pubmed_id'])].extend(get_predefined_keywords(pkwtree,paper['title']))
    predefined_keywords[str(paper['pubmed_id'])] = list(set(predefined_keywords[str(paper['pubmed_id'])]))
len(predefined_keywords)

Wall time: 24.5 s


4749

In [20]:
%%time
mesh_keywrds_extracted = paper_keywords_mesh(articles)
expanded_keywords = get_expanded_keywords(mesh_database, mesh_keywrds_extracted)

len(expanded_keywords)

Wall time: 615 ms


4749

In [22]:
len(abstract_keywords),len(related_keywords),len(predefined_keywords),len(expanded_keywords)

(4749, 4749, 4749, 4749)

In [23]:
extracted_keywords_list  = {}
for key in abstract_keywords.keys():
    extracted_keywords_list[key] = abstract_keywords[key]
    extracted_keywords_list[key].extend(related_keywords[key])
    extracted_keywords_list[key].extend(predefined_keywords[key])
    extracted_keywords_list[key].extend(expanded_keywords[key]) if key in expanded_keywords else extracted_keywords_list[key].extend([])

In [24]:
all_keywords = []
for key,value in extracted_keywords_list.items():
    all_keywords.extend(value)

In [81]:
from collections import Counter
freq_keywords  = Counter(all_keywords)
ranked_keywords = dict(sorted(freq_keywords.items(), key=lambda item: item[1],reverse=True))

keywords_scores = {}
for key,value in ranked_keywords.items():
    if value/len(abstract_keywords) > 0.10:
        keywords_scores[key] = value/len(abstract_keywords)
    
keywords_scores

{'Id': 0.87597388923984,
 'taxonomy': 0.8372288902926932,
 'Immunology': 0.8368077489997894,
 'secretion': 0.8368077489997894,
 'systematics': 0.8365971783533376,
 'hierarchy': 0.8365971783533376,
 'hierarchies': 0.8365971783533376,
 'Classification': 0.8365971783533376,
 'degradation': 0.8363866077068857,
 'catabolism': 0.8363866077068857,
 'biochemical pathways': 0.8361760370604338,
 'biodegradation': 0.8361760370604338,
 'Metabolism': 0.8361760370604338,
 'biotransformation': 0.8361760370604338,
 'Radiation Effects': 0.8349126131817225,
 'chemical properties': 0.833859759949463,
 'heredity': 0.833859759949463,
 'Genetics': 0.833859759949463,
 'chemical composition': 0.833859759949463,
 'Chemistry': 0.833859759949463,
 'chemical content': 0.833859759949463,
 'chemical characterization': 0.833859759949463,
 'chemical structure': 0.833859759949463,
 'historical aspects': 0.8307012002526848,
 'History': 0.8307012002526848,
 'historical notes': 0.8307012002526848,
 'fees': 0.830280058959

In [82]:
len(keywords_scores)

382

In [315]:
# import matplotlib.pyplot as plt
# from wordcloud import WordCloud, STOPWORDS
# comment_words = ' '
# stopwords = set(STOPWORDS) 
# wc = WordCloud(background_color="white",stopwords=stopwords, contour_width=10, contour_color='steelblue')
# wordcloud = wc.generate(" ".join(matches))

# # Display the generated image:
# plt.figure(figsize=(18,10))
# plt.imshow(wordcloud, interpolation='bilinear')
# plt.axis("off")